In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fft2, ifft2, fftfreq
from scipy.signal import windows

In [182]:
# Parametry siatki
L = 10.0           # długość boku obszaru
N = 2**8            # liczba punktów siatki w każdym kierunku
dx = L / N         # krok przestrzenny
x = np.linspace(-L/2, L/2, N, endpoint=False)
y = np.linspace(-L/2, L/2, N, endpoint=False)
X, Y = np.meshgrid(x, y, indexing='ij')

# Parametry fizyczne
ħ = 1.0
m = 1.0
dt = 0.01         # krok czasowy
T_max = 100.0       # maksymalny czas
Nt = int(T_max / dt)

# Potencjał: V(x, y) = x^2 + y^2 + xy
a = 0.001
V = np.sqrt(X**2 + Y**2 + a**2)

# Początkowa funkcja falowa — funkcja Gaussa
sigma = 6.0
ψ = np.exp(-(X**2 + Y**2 - 10) / (2 * sigma**2)) + np.exp(-(X**2 + Y**2 + 3) / (2 * sigma**2))
ψ /= np.linalg.norm(ψ) * dx * dx  # normalizacja

ψ0 = ψ.copy()  # zapisz funkcję początkową do korelacji

In [183]:
# FFT: przygotowanie współrzędnych pędu
kx = 2 * np.pi * fftfreq(N, dx)
ky = 2 * np.pi * fftfreq(N, dx)
KX, KY = np.meshgrid(kx, ky, indexing='ij')
K2 = KX**2 + KY**2

# Fazy propagacji
T_phase = np.exp(-1j * (ħ**2 * K2 / (2 * m)) * dt / (2 * ħ))
V_phase = np.exp(-1j * V * dt / ħ)

In [ ]:
# Rejestracja g1(t)
g1_t = []

# Pętla czasowa
for _ in range(Nt):
    # pół krok w pędzie
    ψ̃ = fft2(ψ)
    ψ̃ *= T_phase
    ψ = ifft2(ψ̃)

    # pełen krok w przestrzeni
    ψ *= V_phase

    # drugi pół krok w pędzie
    ψ̃ = fft2(ψ)
    ψ̃ *= T_phase
    ψ = ifft2(ψ̃)

    # oblicz g1(t)
    g1 = np.sum(np.conj(ψ0) * ψ) * dx * dx
    g1_t.append(g1)

# Czas i amplituda korelacji
t_values = np.arange(Nt) * dt
g1_t = np.array(g1_t)
amplitude = np.abs(g1_t)

In [ ]:
# Wykres
plt.figure(figsize=(10, 4))
plt.plot(t_values, amplitude)
plt.xlabel("Czas t")
plt.ylabel(r"$|g_1(t)|$")
plt.title("Moduł funkcji korelacyjnej $g_1(t)$")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
T = len(g1_t) * dt             # cały czas symulacji
t = np.linspace(0, T, len(g1_t), endpoint = False)

# Stosujemy funkcję okna Hann do g1(t)
window = window = (1 - np.cos(2 * np.pi * t / T)) / T
g1_windowed = g1_t * window

# Transformata Fouriera z oknem
g1_E = fft(g1_windowed)
freqs = fftfreq(Nt, dt)  # jednostki częstotliwości (odwrotność czasu)
E = 2 * np.pi * freqs * ħ  # przekształcenie na energię

# Przesuwamy i wybieramy tylko dodatnie energie
E = np.fft.fftshift(E)
g1_E = np.fft.fftshift(g1_E)

# Wykres widma
plt.figure(figsize=(10, 4))
plt.plot(E, np.abs(g1_E) * 2 / len(g1_E), lw=1)
plt.xlim(-10, 10)  # ogranicz zakres energii do rozsądnego przedziału
plt.xlabel("Energia E")
plt.ylabel(r"$|g_1(E)|$")
plt.title("Widmo energii uzyskane przez FFT z oknem Hann")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
from scipy.optimize import curve_fit

# Funkcja pojedynczej linii spektralnej (np. Gauss z szerokością gamma i środkiem E0)
def spectral_line(E, A, E0, gamma):
    return A * np.exp(-((E - E0) ** 2) / (2 * gamma ** 2))

# Szukamy lokalnych maksimów jako zgadywanek początkowych (proste podejście: pik powyżej progu)
threshold = np.max(np.abs(g1_E)) * 0.2
peaks = (np.abs(g1_E)[1:-1] > np.abs(g1_E)[:-2]) & (np.abs(g1_E)[1:-1] > np.abs(g1_E)[2:]) & (np.abs(g1_E)[1:-1] > threshold)
peak_indices = np.where(peaks)[0] + 1
peak_energies = E[peak_indices]
peak_amplitudes = np.abs(g1_E)[peak_indices]

# Dopasowanie kilku linii spektralnych
def multi_spectral_line(E, *params):
    result = np.zeros_like(E)
    for i in range(0, len(params), 3):
        A, E0, gamma = params[i:i+3]
        result += spectral_line(E, A, E0, gamma)
    return result

# Początkowe zgadywanki parametrów: amplituda, pozycja, szerokość
initial_guess = []
for amp, pos in zip(peak_amplitudes, peak_energies):
    initial_guess += [amp, pos, 0.1]

# Dopasowanie
popt, _ = curve_fit(multi_spectral_line, E, np.abs(g1_E), p0=initial_guess, maxfev=10000)

# Wykres porównawczy
plt.figure(figsize=(10, 4))
plt.plot(E, np.abs(g1_E), label="Widmo FFT", lw=1)
plt.plot(E, multi_spectral_line(E, *popt), label="Dopasowanie", lw=2)
plt.xlim(-10,10)
plt.xlabel("Energia E")
plt.ylabel(r"$|g_1(E)|$")
plt.title("Dopasowanie linii spektralnych metodą najmniejszych kwadratów")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Wyodrębnij dopasowane wartości energii
eigenvalues = [popt[i+1] for i in range(0, len(popt), 3)]
eigenvalues


In [ ]:
# Parametry siatki
L = 10.0           # długość boku obszaru
N = 2**9            # liczba punktów siatki w każdym kierunku
dx = L / N         # krok przestrzenny
x = np.linspace(-L/2, L/2, N, endpoint=False)

# Parametry fizyczne
ħ = 1.0
m = 0.5
dt = 5.73         # krok czasowy
T_max = 5.0       # maksymalny czas
Nt = int(T_max / dt)

# Potencjał: V(x, y) = x^2 + y^2 + xy
k0 = -132.7074997
k2 = 7
k3 = 0.5
k4 = 1
V = k0 - k2 * x**2 + k3 * x**3 + k4 * x**4 

# Początkowa funkcja falowa — funkcja Gaussa
a = 1.9
sigma = 0.87
ψ = np.exp(-(x - a)**2 / (2 * sigma**2)) + np.exp(-(x + a)**2 / (2 * sigma**2))
ψ /= np.linalg.norm(ψ) * dx * dx  # normalizacja

ψ0 = ψ.copy()  # zapisz funkcję początkową do korelacji